In [9]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\general_functions\Extracting Data Functions.ipynb"

#### Data Parameters

In [10]:
f_name = os.path.join(os.path.dirname(os.getcwd()),"Edgewell US Male Dataset.xlsx")
ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
prodORitem="SKU"

client_manuf = ["Pbg"]
client_brands = ["Schick", "Equate", "Cremo"]

categories = ["Manual Shave Men"]
sectors = ["System","Disposables"]
segments = ["Razors", "Refills", "Disposables"]
subsegments= []
subcategories= []

decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

customareas='REGIONS'
national = False
areas = ["RETAILER",f"{customareas}"]
regions_RET  =["Walmart"]
channels_RET = []
market_RET = ["Walmart Div1 Corp","Walmart Nm Corp","Walmart Sc Corp"]

regions_CHAN = []
channels_CHAN = []
market_CHAN = []

regions_CUST = []
channels_CUST = []
market_CUST = ["Walmart East","Walmart North","Walmart Southeast","Walmart Southwest","Walmart West"]


data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending March  2025"
end_date = "2025-04-01"

years = ['2023', '2024', '2025']
subcatg_parent = "Segment" #still used?
subcatg_parent_list = segments #still used?
 
percent = 1000000
percentstr="'000 000"
 
normalized = True
promo_type = True
dispaly_share = False  # True if Available
feature_share = False

 
direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories)),
                "Segment":list(zip(["Sector"]*len(sectors),sectors)),
                "SubSegment":list(zip(["Segment"]*len(segments),segments)),
                "SubCategory":list(zip(["Segment"]*len(segments),segments))}

In [11]:
# filter_dictionary_keys(fieldsNamePosition, "Promo Type")
# promo_col

In [12]:
month_years =  pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST
promo_col = []
promo_col = promo_col+['[Measures].[Display Share]'] if dispaly_share else promo_col
promo_col = promo_col+['[Measures].[Feature Share]'] if feature_share else promo_col

In [14]:
f_path = Path.cwd()
excel = client.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True  # False
wb = excel.Workbooks.Open(f_name)
ws=wb.Sheets([s.Name for s in wb.Sheets][0])
pvtTable = ws.PivotTables(1)
s_name = [s.Name for s in wb.Sheets][0]

#change report layout
pvtTable.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form

#change pivot table style
#Select from Design tab, try out Medium9 or Medium3
pvtTable.TableStyle2 = "pivotStyleMedium21"
pvtTable.ClearTable()

pvtTable.TableRange2.Cut(ws.Range("A16"))

fieldsNamePosition={}

for i in range(1,pvtTable.CubeFields.Count+1):
    fieldsNamePosition[str(pvtTable.CubeFields(i))]=i

In [ ]:
# filter_dictionary_keys(fieldsNamePosition, "Promo Sale")

In [ ]:
start_time = time.time()

## By Promo Types

#### Brands and Promo Types

In [15]:
brands_promo_type = {}

In [16]:
if promo_type:
    row_list=[f'[Products].[{BrandOrTopB}]','[Promo Description].[Promo Type]'] 
    column_list=[]
    filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list = ['[Measures].[Value Share]',  '[Measures].[Promo Sales]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    if len(categories) != 0:
        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in month_years:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 
    if len(categories)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        

In [17]:
if promo_type:    
    if len(categories)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_data(brands_promo_type,f'{categories[0]} | ')
    # By Sector

    if len(sectors)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
        calculate_data(brands_promo_type, '',sectors,'[Products].[Sector].[Sector]')

    if len(segments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
        calculate_data(brands_promo_type, '',segments,'[Products].[Segment].[Segment]')

    if len(subsegments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
        calculate_data(brands_promo_type, '',subsegments,'[Products].[SubSegment].[SubSegment]')

    if len(subcategories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
        calculate_data(brands_promo_type, '',subcategories,'[Products].[SubCategory].[SubCategory]')
        
with open('Promotion Datasets/brands_promo_type.pickle', 'wb') as handle:
    pickle.dump(brands_promo_type, handle, protocol=pickle.HIGHEST_PROTOCOL)  
    


#### By Brands For P12M

In [18]:
promotions_brands_P12M = {}

In [19]:
row_list=[f'[Products].[{BrandOrTopB}]', '[Products].[Total Size]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list = []

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in month_years:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 
if len(categories)!=0 :
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    

In [20]:

if normalized == True:
    value_list=['[Measures].[Promo Value]', '[Measures].[VSOD]', '[Measures].[VSOD IYA]',  '[Measures].[Value Share]','[Measures].[Promo Share]', '[Measures].[Value Uplift (v. base) Normalized]', '[Measures].[Value Uplift Normalized IYA]','[Measures].[Volume Uplift (v. Base) Normalized]', '[Measures].[Volume Uplift Normalized IYA]'] + promo_col
else:
    value_list=['[Measures].[Promo Value]', '[Measures].[VSOD]', '[Measures].[VSOD IYA]',  '[Measures].[Value Share]','[Measures].[Promo Share]', '[Measures].[Value Uplift (v. base)]', '[Measures].[Value Uplift IYA]','[Measures].[Volume Uplift (v. Base)]', '[Measures].[Volume Uplift IYA]'] + promo_col
    
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    
if len(categories)!=0 :
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(promotions_brands_P12M,f'{categories[0]} | ')
# By Sector

if len(sectors)!=0 :
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(promotions_brands_P12M, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(promotions_brands_P12M, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(promotions_brands_P12M, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(promotions_brands_P12M, '',subcategories,'[Products].[SubCategory].[SubCategory]')
    
with open('Promotion Datasets/promotions_brands_P12M.pickle', 'wb') as handle:
    pickle.dump(promotions_brands_P12M, handle, protocol=pickle.HIGHEST_PROTOCOL)  
    


# VSOD

In [21]:
for key_child, val_child in direct_parent.items():
    globals()[f"{key_child}_VSOD"] = {}
        
    row_list=[f'[Products].[{val_child[0][0]}]',f'[Products].[{key_child}]']
    column_list=[]
    filter_list=['[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    value_list=['[Measures].[VSOD]']
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


    pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
    visible_items = []
    for value in month_years:
        visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
    pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items

    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(globals()[f"{key_child}_VSOD"],f'{categories[0]} | ')


    with open(f'Promotion Datasets/{key_child}_VSOD.pickle', 'wb') as handle:
        pickle.dump(globals()[f"{key_child}_VSOD"], handle, protocol=pickle.HIGHEST_PROTOCOL)



### VSOD Client

In [22]:
#### Sector
if len(sectors)>0:  
    direct_secparent = {key: value for key, value in direct_parent.items() if key == "Sector"}
    
    for key_child, val_child in direct_secparent.items():
        print(key_child,[i[1] for i in val_child],val_child[0][0])
        # for key_par, val_par in val_child.items():
        globals()[f"{key_child}_client_VSOD"] = {}
        globals()[f"{key_child}_manuf_VSOD"] = {}
        if len(client_brands)!=0:
            row_list=[f'[Products].[{val_child[0][0]}]',f'[Products].[{key_child}]']
            column_list=[f'[Products].[{BrandOrTopB}]']
            filter_list=['[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
            value_list=['[Measures].[VSOD]']

            pvtTable.ClearTable()
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


            pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
            pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
            visible_items = []
            for value in month_years:
                visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
            pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 


            pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
            pvtTable.CubeFields(f'[Products].[{BrandOrTopB}]').EnableMultiplePageItems = True 
            visible_items = []
            for value in client_brands:
                visible_items.append(f"[Products].[{BrandOrTopB}].&[{value}]")
            pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').VisibleItemsList = visible_items 

            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
            calculate_category_data( globals()[f"{key_child}_client_VSOD"])
                
            with open(f'Promotion Datasets/{key_child}_client_VSOD.pickle', 'wb') as handle:
                    pickle.dump(globals()[f"{key_child}_client_VSOD"], handle, protocol=pickle.HIGHEST_PROTOCOL)
        if len(client_manuf)!=0:
            row_list=[f'[Products].[{val_child[0][0]}]',f'[Products].[{key_child}]']
            column_list=[f'[Products].[{ManufOrTopC}]']
            filter_list=['[Scope].[Scope]','[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
            value_list=['[Measures].[VSOD]']

            pvtTable.ClearTable()
            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
            pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
            visible_items = []
            for value in month_years:
                visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
            pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
            
            pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').ClearAllFilters()
            pvtTable.CubeFields(f'[Products].[{ManufOrTopC}]').EnableMultiplePageItems = True 
            visible_items = []
            for value in client_manuf:
                visible_items.append(f"[Products].[{ManufOrTopC}].&[{value}]")
            pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').VisibleItemsList = visible_items 

            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
            calculate_category_data(globals()[f"{key_child}_manuf_VSOD"])

            with open(f'Promotion Datasets/{key_child}_manuf_VSOD.pickle', 'wb') as handle:
                    pickle.dump(globals()[f"{key_child}_manuf_VSOD"], handle, protocol=pickle.HIGHEST_PROTOCOL)

Sector ['Manual Shave Men'] Category


In [23]:
#### segment, subsegment,subcategory 
for key_child, val_child in direct_parent.items():
    if val_child[0][0] == "Category":
        continue  # Skip this iteration and continue to the next one

    print(key_child,[i[1] for i in val_child],val_child[0][0])
    # for key_par, val_par in val_child.items():
    globals()[f"{key_child}_client_VSOD"] = {}
    globals()[f"{key_child}_manuf_VSOD"] = {}
    if len(client_brands)!=0:
        row_list=[f'[Products].[{val_child[0][0]}]',f'[Products].[{key_child}]']
        column_list=[f'[Products].[{BrandOrTopB}]']
        filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
        value_list=['[Measures].[VSOD]']

        pvtTable.ClearTable()
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

        if len(categories) != 0:
            pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


        pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
        pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
        visible_items = []
        for value in month_years:
            visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
        pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 


        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').ClearAllFilters()
        pvtTable.CubeFields(f'[Products].[{BrandOrTopB}]').EnableMultiplePageItems = True 
        visible_items = []
        for value in client_brands:
            visible_items.append(f"[Products].[{BrandOrTopB}].&[{value}]")
        pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').VisibleItemsList = visible_items 

        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_category_data( globals()[f"{key_child}_client_VSOD"])
            
        with open(f'Promotion Datasets/{key_child}_client_VSOD.pickle', 'wb') as handle:
                pickle.dump(globals()[f"{key_child}_client_VSOD"], handle, protocol=pickle.HIGHEST_PROTOCOL)
    if len(client_manuf)!=0:
        row_list=[f'[Products].[{val_child[0][0]}]',f'[Products].[{key_child}]']
        column_list=[f'[Products].[{ManufOrTopC}]']
        filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
        value_list=['[Measures].[VSOD]']

        pvtTable.ClearTable()
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

        if len(categories) != 0:
            pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

        pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
        pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
        visible_items = []
        for value in month_years:
            visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
        pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items
        
        pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').ClearAllFilters()
        pvtTable.CubeFields(f'[Products].[{ManufOrTopC}]').EnableMultiplePageItems = True 
        visible_items = []
        for value in client_manuf:
            visible_items.append(f"[Products].[{ManufOrTopC}].&[{value}]")
        pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').VisibleItemsList = visible_items 

        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_category_data(globals()[f"{key_child}_manuf_VSOD"])

        with open(f'Promotion Datasets/{key_child}_manuf_VSOD.pickle', 'wb') as handle:
                pickle.dump(globals()[f"{key_child}_manuf_VSOD"], handle, protocol=pickle.HIGHEST_PROTOCOL)




Segment ['System', 'Disposables'] Sector
SubSegment ['Razors', 'Refills', 'Disposables'] Segment
SubCategory ['Razors', 'Refills', 'Disposables'] Segment


#### By Products/Item P12M

In [24]:
promotions_products_P12M = {}

In [25]:
row_list=[f'[Products].[{BrandOrTopB}]', f'[Products].[{prodORitem}]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]', '[Calendar].[MonthYear]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
if normalized == True:
    value_list=['[Measures].[Promo Sales]', '[Measures].[Promo Value]','[Measures].[Discount Depth (%)]', '[Measures].[Promo Share]','[Measures].[VSOD]','[Measures].[Base Price/Unit]','[Measures].[Promo Price/Unit]','[Measures].[Gross Margin %]','[Measures].[Value Uplift (v. base) Normalized]', '[Measures].[Volume Uplift (v. Base) Normalized]','[Measures].[Trade Effectiveness]','[Measures].[Incr Value]']
else:
    value_list=['[Measures].[Promo Sales]', '[Measures].[Promo Value]','[Measures].[Discount Depth (%)]', '[Measures].[Promo Share]','[Measures].[VSOD]','[Measures].[Base Price/Unit]','[Measures].[Promo Price/Unit]','[Measures].[Gross Margin %]','[Measures].[Value Uplift (v. base)]', '[Measures].[Volume Uplift (v. Base)]','[Measures].[Trade Effectiveness]','[Measures].[Incr Value]']
    
pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[MonthYear]').EnableMultiplePageItems = True 
visible_items = []
for value in month_years:
    visible_items.append(f"[Calendar].[MonthYear].&[{value}]")
pvtTable.PivotFields('[Calendar].[MonthYear].[MonthYear]').VisibleItemsList = visible_items 

In [26]:
if len(categories)!=0 :
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_data(promotions_products_P12M,f'{categories[0]} | ')
# By Sector

if len(sectors)!=0 :
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
    calculate_data(promotions_products_P12M, '',sectors,'[Products].[Sector].[Sector]')

if len(segments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
    calculate_data(promotions_products_P12M, '',segments,'[Products].[Segment].[Segment]')

if len(subsegments)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
    calculate_data(promotions_products_P12M, '',subsegments,'[Products].[SubSegment].[SubSegment]')

if len(subcategories)!=0:
    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    calculate_data(promotions_products_P12M, '',subcategories,'[Products].[SubCategory].[SubCategory]')
    
with open('Promotion Datasets/promotions_products_P12M.pickle', 'wb') as handle:
    pickle.dump(promotions_products_P12M, handle, protocol=pickle.HIGHEST_PROTOCOL)  

### By End of The Week

In [27]:
promotions_EndOfWeek = {}

In [28]:
row_list=['[Calendar].[End of Week]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{ManufOrTopC}]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list = []

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

In [29]:
if normalized == True:
    value_list=['[Measures].[Value Uplift (v. base) Normalized]','[Measures].[Promo Value]','[Measures].[Non Promo Value]','[Measures].[Value Sales]','[Measures].[Base Sales]','[Measures].[VSOD]','[Measures].[Promo Volume]','[Measures].[Non Promo Volume]']
else:
    value_list=['[Measures].[Value Uplift (v. base)]','[Measures].[Promo Value]','[Measures].[Non Promo Value]','[Measures].[Value Sales]','[Measures].[Base Sales]','[Measures].[VSOD]','[Measures].[Promo Volume]','[Measures].[Non Promo Volume]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)



for manuf in client_manuf:
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').CurrentPageName = f'[Products].[{ManufOrTopC}].&[{manuf}]'
    if len(categories)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_data(promotions_EndOfWeek,f'{categories[0]} | {manuf} | ')
    # By Sector

    if len(sectors)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
        calculate_data(promotions_EndOfWeek,f'{manuf} | ',sectors,'[Products].[Sector].[Sector]')

    if len(segments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
        calculate_data(promotions_EndOfWeek,f'{manuf} | ',segments,'[Products].[Segment].[Segment]')

    if len(subsegments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
        calculate_data(promotions_EndOfWeek,f'{manuf} | ',subsegments,'[Products].[SubSegment].[SubSegment]')

    if len(subcategories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
        calculate_data(promotions_EndOfWeek, f'{manuf} | ',subcategories,'[Products].[SubCategory].[SubCategory]')
        
  

In [30]:
row_list=['[Calendar].[End of Week]']
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
if normalized == True:
    value_list=['[Measures].[Value Uplift (v. base) Normalized]','[Measures].[Promo Value]','[Measures].[Non Promo Value]','[Measures].[Value Sales]','[Measures].[Base Sales]','[Measures].[VSOD]','[Measures].[Promo Volume]','[Measures].[Non Promo Volume]']
else:
    value_list=['[Measures].[Value Uplift (v. base)]','[Measures].[Promo Value]','[Measures].[Non Promo Value]','[Measures].[Value Sales]','[Measures].[Base Sales]','[Measures].[VSOD]','[Measures].[Promo Volume]','[Measures].[Non Promo Volume]']


pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

In [31]:
for brand in client_brands:
    pvtTable.PivotFields(f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]").ClearAllFilters()
    pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'

    if len(categories)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_data(promotions_EndOfWeek,f'{categories[0]} | {brand} | ')
# By Sector
    if len(sectors)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
        calculate_data(promotions_EndOfWeek,f'{brand} | ',sectors,'[Products].[Sector].[Sector]')

    if len(segments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
        calculate_data(promotions_EndOfWeek,f'{brand} | ',segments,'[Products].[Segment].[Segment]')

    if len(subsegments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
        calculate_data(promotions_EndOfWeek,f'{brand} | ',subsegments,'[Products].[SubSegment].[SubSegment]')

    if len(subcategories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
        calculate_data(promotions_EndOfWeek, f'{brand} | ',subcategories,'[Products].[SubCategory].[SubCategory]')
        

In [32]:
with open('Promotion Datasets/promotions_EndOfWeek.pickle', 'wb') as handle:
    pickle.dump(promotions_EndOfWeek, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Value Uplift vs Discount depth By  Client Brand

In [33]:
value_uplift = {}

In [34]:
row_list=[]
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{BrandOrTopB}]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=[]

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

row_list = [f'[Products].[{prodORitem}]','[Calendar].[End of Week]']
if normalized == True:
    value_list = ['[Measures].[Discount Depth (%)]','[Measures].[Promo Price/Unit]', '[Measures].[Value Uplift (v. base) Normalized]','[Measures].[Promo Sales]']
else:
    value_list = ['[Measures].[Discount Depth (%)]','[Measures].[Promo Price/Unit]', '[Measures].[Value Uplift (v. base)]','[Measures].[Promo Sales]']

pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


In [35]:
for brand in client_brands:
    pvtTable.PivotFields(f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]").ClearAllFilters()
    pvtTable.PivotFields(f'[Products].[{BrandOrTopB}].[{BrandOrTopB}]').CurrentPageName = f'[Products].[{BrandOrTopB}].&[{brand}]'
   
    if len(categories) != 0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        if len(categories)!=0 :
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
            calculate_data(value_uplift,f'{categories[0]} | {brand} | ')
    # By Sector
        if len(sectors)!=0 :
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
            calculate_data(value_uplift,f'{brand} | ',sectors,'[Products].[Sector].[Sector]')

        if len(segments)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
            calculate_data(value_uplift,f'{brand} | ',segments,'[Products].[Segment].[Segment]')

        if len(subsegments)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
            calculate_data(value_uplift,f'{brand} | ',subsegments,'[Products].[SubSegment].[SubSegment]')

        if len(subcategories)!=0:
            pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
            pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
            calculate_data(value_uplift, f'{brand} | ',subcategories,'[Products].[SubCategory].[SubCategory]')
            

In [36]:
with open('Promotion Datasets/value_uplift.pickle', 'wb') as handle:
    pickle.dump(value_uplift, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Value Uplift vs Discount depth By  Client Manuf

In [37]:
value_uplift_manuf = {}

In [38]:
row_list=[]
column_list=[]
filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{ManufOrTopC}]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=[]

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

if len(categories) != 0:
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'
    # pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

row_list = ['[Products].[Product]','[Calendar].[End of Week]']
if normalized == True:
    value_list = ['[Measures].[Discount Depth (%)]','[Measures].[Promo Price/Unit]', '[Measures].[Value Uplift (v. base) Normalized]','[Measures].[Promo Sales]']
else:
    value_list = ['[Measures].[Discount Depth (%)]','[Measures].[Promo Price/Unit]', '[Measures].[Value Uplift (v. base)]','[Measures].[Promo Sales]']

pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


In [39]:
for manuf in client_manuf:
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').CurrentPageName = f'[Products].[{ManufOrTopC}].&[{manuf}]'
    if len(categories)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
        calculate_data(value_uplift_manuf,f'{categories[0]} | {manuf} | ')
    # By Sector

    if len(sectors)!=0 :
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'
        calculate_data(value_uplift_manuf,f'{manuf} | ',sectors,'[Products].[Sector].[Sector]')

    if len(segments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'
        calculate_data(value_uplift_manuf,f'{manuf} | ',segments,'[Products].[Segment].[Segment]')

    if len(subsegments)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'
        calculate_data(value_uplift_manuf,f'{manuf} | ',subsegments,'[Products].[SubSegment].[SubSegment]')

    if len(subcategories)!=0:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
        calculate_data(value_uplift_manuf, f'{manuf} | ',subcategories,'[Products].[SubCategory].[SubCategory]')
        

In [40]:
with open('Promotion Datasets/value_uplift_manuf.pickle', 'wb') as handle:
    pickle.dump(value_uplift_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Seasonality index dic

In [41]:
for ke, val in direct_parent.items():
    globals()[f"{ke}_MonthYear"] = {}
    
    row_list=[ f'[Products].[{ke}]','[Calendar].[MonthYear]']
    column_list=[]
    if val[0][0] == "Category":
        filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
    else:
        filter_list=['[Products].[Category]','[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]',f'[Products].[{val[0][0]}]']

    value_list=['Value Sales']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    ### The All brands (need only the Sectors Total rows)
    if val[0][0] == "Category":
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[Category]'
        calculate_data(globals()[f"{ke}_MonthYear"],'')

    else:
        pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = f'[Scope].[Scope].&[{val[0][0]}]'
        calculate_data(globals()[f"{ke}_MonthYear"], '',[i[1] for i in val],f'[Products].[{val[0][0]}].[{val[0][0]}]')


    with open(f'Promotion Datasets/{ke}_MonthYear.pickle', 'wb') as handle:
        pickle.dump(globals()[f"{ke}_MonthYear"], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
Category_MonthYear={}
row_list=[ f'[Products].[Category]','[Calendar].[MonthYear]']
column_list=[]
filter_list=[f"[Products].[{ManufOrTopC}]",'[Scope].[Scope]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']

value_list=['Value Sales']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


if len(client_manuf)>1:
    pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
else:
    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
calculate_data(Category_MonthYear,f'{client_manuf[0]} | ')

with open(f'Promotion Datasets/Category_MonthYear.pickle', 'wb') as handle:
    pickle.dump(Category_MonthYear, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [43]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")


NameError: name 'start_time' is not defined